In [1]:
import tensorflow as tf
from tensorflow import keras
import json



In [2]:
from data_utils import Data


config = json.load(open("./config.json"))

# Load training data
training_data = Data(data_source=config["data"]["training_data_source"], alphabet=config["data"]["alphabet"],
                         input_size=config["data"]["input_size"],num_of_classes=config["data"]["num_of_classes"])
training_data.load_data()
training_inputs, training_labels, batch_texts = training_data.get_all_data()


# Load validation data
validation_data = Data(data_source=config["data"]["validation_data_source"], alphabet=config["data"]["alphabet"],
                           input_size=config["data"]["input_size"], num_of_classes=config["data"]["num_of_classes"])
validation_data.load_data()
validation_inputs, validation_labels, batch_texts = validation_data.get_all_data()

# Load test data
test_data = Data(data_source=config["data"]["test_data_source"], alphabet=config["data"]["alphabet"],
                           input_size=config["data"]["input_size"], num_of_classes=config["data"]["num_of_classes"])
test_data.load_data()
test_inputs, test_labels, batch_texts = test_data.get_all_data()


Data loaded from ./_train.csv
Data loaded from ./_validate.csv
Data loaded from ./_test.csv


In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import csv
from tensorflow.keras.layers import Embedding, Bidirectional,LSTM,Dense,Dropout
from itertools import chain

df = pd.read_csv('_train.csv')

from tensorflow.keras.layers import TextVectorization


text_data = []
with open('_train.csv', 'r', newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        text_data.extend(row)  # Extend the list instead of appending each row

# Flatten the nested list
text_data_flat = [item for sublist in text_data for item in sublist]





"""text_data = []
with open('_train.csv', 'r', newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        text_data.extend(row)  # Extend the list instead of appending each row

# Flatten the nested list and convert it to a single string
text_data_flat = ' '.join(text_data)"""



# Define the TextVectorization layer
encoder = TextVectorization(max_tokens=1000, output_mode='int')

# Manually preprocess the text data
encoder.adapt([text_data_flat])


# Example usage:
# encoded_text = encoder(text_data)


model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])


model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model.summary()

ValueError: When using `TextVectorization` to tokenize strings, the input rank must be 1 or the last shape dimension must be 1. Received: inputs.shape=(1, 33760937) with rank=2

In [ ]:
# Train

epochs=config["training"]["epochs"]
batch_size=config["training"]["batch_size"]
checkpoint_every=config["training"]["checkpoint_every"]
    
    
print("Training model: ")
model.fit(training_inputs, training_labels, validation_data=(validation_inputs, validation_labels),
          epochs=epochs, batch_size=batch_size, verbose=2, callbacks=[])

#model.fit(training_inputs, training_labels,
#          epochs=epochs, batch_size=batch_size, verbose=2, callbacks=[])


# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save("modelRNN.h5")
print("Saved model to disk")

Training model: 
Epoch 1/7


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 5), output.shape=(None, 1)

In [1]:
# load json and create model
from keras.models import model_from_json
import keras_metrics

json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("modelRNN.weights.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(optimizer=optimizer, loss=loss, metrics=[tensorflow.keras.metrics.Precision(), tensorflow.keras.metrics.Recall()])
score = loaded_model.evaluate(test_inputs, test_labels, batch_size=batch_size, verbose=1)

for i in range(0,len(loaded_model.metrics_names)):
    print("%s: %.2f%%" % (loaded_model.metrics_names[i], score[i]*100))

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'modelRNN.weights.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

model = loaded_model
y_test = test_labels
Y_test = np.argmax(y_test, axis=1) # Convert one-hot to index
y_pred = model.predict(test_inputs)
y_pred = np.argmax(y_pred,axis=-1)
#print(classification_report(Y_test, y_pred))
class_mapping = {0: "not anomaly", 1: "anomaly"}

# Map integer labels to class names
classified_labels = [class_mapping[label] for label in y_pred]

# Now, classified_labels contains the class names for each sample, classified as "anomaly" or "not anomaly"
print(classified_labels)
# for ECMLPKDD
# valid = 0
# xss =  1
# sqlinjection = 2
# ldapinjection  = 3
# xpathinjection  = 4
# pathtransversal = 5
# oscommanding  = 6
# ssi = 7

# for  CISC
# valid = 0
# malicious = 1

# for Morzeux_HttpParamsDataset
# valid = 0
# sqli  = 1
# xss   = 2
# path-traversal = 3
# cmdi = 4

607/607 ━━━━━━━━━━━━━━━━━━━━ 37s 60ms/step
['not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'anomaly', 'anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not anomaly', 'not 